In [1]:
from glob import glob
import os
from tqdm import tqdm
import torch
import torchvision
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
%matplotlib inline



/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [3]:
w, h = 20, 20
base_path = os.path.join('./dataset/', '*.npz')

file_list = glob(base_path)

x_data, y_data = [], []
cut = 0
for file_path in tqdm(file_list):
  data = np.load(file_path)
  x_data.extend(data['inputs'])
  y_data.extend(data['outputs'])
  # too many data for google colab, so I've only use 5000
  if cut == 1000:   
    break
  else:
    cut += 1

# It's faster to convert np.array to tensor rather than conver np.ndarray to tensor directly
x_data = torch.tensor(np.array(x_data, np.float32)).reshape((-1, h, w, 1))
y_data = torch.tensor(np.array(y_data, np.float32)).reshape((-1, h * w))


  4%|▍         | 1000/22536 [00:08<03:08, 114.28it/s]


In [5]:
# Set DataSet
full_dataset = TensorDataset(x_data, y_data)

del x_data, y_data

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size

#split datasets
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])


# Define data loader0
learning_rate = 0.001
training_epochs = 15
batch_size = 100
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [14]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    # L1 ImgIn shape=(?, 20, 20, 1)
    #    Conv     -> (?, 20, 20, 64)
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 64, kernel_size=7, stride=1, padding="same"),
        torch.nn.ReLU())
    
    # L2 ImgIn shape=(?, 20, 20, 64)
    #    Conv     -> (?, 20, 20, 128)
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(64, 128, kernel_size=7, stride=1, padding="same"),
        torch.nn.ReLU())
    
    # L3 ImgIn shape=(?, 20, 20, 128)
    #    Conv     -> (?, 20, 20, 256)
    self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(128, 256, kernel_size=7, stride=1, padding="same"),
        torch.nn.ReLU())
    
    # L4 ImgIn shape=(?, 20, 20, 256)
    #    Conv     -> (?, 20, 20, 128)
    self.layer4 = torch.nn.Sequential(
        torch.nn.Conv2d(256, 128, kernel_size=7, stride=1, padding="same"),
        torch.nn.ReLU())
    
    # L5 ImgIn shape=(?, 20, 20, 128)
    #    Conv     -> (?, 20, 20, 64)
    self.layer5 = torch.nn.Sequential(
        torch.nn.Conv2d(128, 64, kernel_size=7, stride=1, padding="same"),
        torch.nn.ReLU())
    
    # L6 ImgIn shape=(?, 20, 20, 64)
    #    Conv     -> (?, 20, 20, 1)
    self.layer6 = torch.nn.Sequential(
        torch.nn.Conv2d(64, 1, kernel_size=1, stride=1, padding="same"))

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = torch.nn.Sigmoid(out)

In [15]:
# CNN 모델 정의
model = CNN().to(device)

In [10]:
total_batch = len(train_dl)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 4651


In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
for epoch in range(training_epochs + 1):
    
    avg_cost = 0
    
    for X, y in train_dl:
        X = X.to(device)
        y = y.to(device)
        
        # H(x) 계산
        hypothesis = model.forward(X)
        # cost 계산
        cost = F.binary_cross_entropy(hypothesis, y)
        
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

NotImplementedError: 

In [ ]:
model = models.Sequential([
    layers.Conv2D(64, 7, activation='relu', padding='same', input_shape=(h, w, 1)),
    layers.Conv2D(128, 7, activation='relu', padding='same'),
    layers.Conv2D(256, 7, activation='relu', padding='same'),
    layers.Conv2D(128, 7, activation='relu', padding='same'),
    layers.Conv2D(64, 7, activation='relu', padding='same'),
    layers.Conv2D(1, 1, activation=None, padding='same'),
    layers.Reshape((h * w,)),
    layers.Activation('sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['acc']
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 20, 20, 64)        3200      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 128)       401536    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 256)       1605888   
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 128)       1605760   
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 20, 20, 64)        401472    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 20, 20, 1)         65        
_________________________________________________________________
reshape (Reshape)            (None, 400)               0

- kernel 7,5,3,3,3: 53.3
- kernel all 7: 56.12
- kernel all 7, double kernel:

In [ ]:
start_time = datetime.now().strftime('%Y%m%d_%H%M%S')
os.makedirs('models', exist_ok=True)

model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=10,
    callbacks=[
        ModelCheckpoint('./models/%s.h5' % (start_time), monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, verbose=1, mode='auto')
    ],
    validation_data=(x_val, y_val),
    use_multiprocessing=True,
    workers=16
)

Epoch 1/10
17637/17638 [============================>.] - ETA: 0s - loss: 0.0049 - acc: 0.6081
Epoch 00001: val_acc improved from -inf to 0.58375, saving model to ./models/20201213_202430.h5
17638/17638 [==============================] - 1091s 62ms/step - loss: 0.0049 - acc: 0.6081 - val_loss: 0.0052 - val_acc: 0.5837
Epoch 2/10
17637/17638 [============================>.] - ETA: 0s - loss: 0.0048 - acc: 0.6206
Epoch 00002: val_acc improved from 0.58375 to 0.58817, saving model to ./models/20201213_202430.h5
17638/17638 [==============================] - 1095s 62ms/step - loss: 0.0048 - acc: 0.6206 - val_loss: 0.0052 - val_acc: 0.5882
Epoch 3/10
17637/17638 [============================>.] - ETA: 0s - loss: 0.0046 - acc: 0.6330
Epoch 00003: val_acc improved from 0.58817 to 0.59378, saving model to ./models/20201213_202430.h5
17638/17638 [==============================] - 1096s 62ms/step - loss: 0.0046 - acc: 0.6330 - val_loss: 0.0052 - val_acc: 0.5938
Epoch 4/10
17637/17638 [==========

In [ ]:
i = 0

for y in range(h):
    for x in range(w):
        print('%2d' % x_val[i][y, x], end='')
    print()

 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0-1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0-1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [ ]:
y_pred = model.predict(np.expand_dims(x_val[i], axis=0)).squeeze()
y_pred = y_pred.reshape((h, w))

y, x = np.unravel_index(np.argmax(y_pred), y_pred.shape)

print(x, y, y_pred[y, x])

6 9 0.99919564
